<a href="https://colab.research.google.com/github/paridaabhishek/Alpacatrade/blob/master/ema9andema21Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This process will try to figure out the net gain when we automate the ema 9 and ema 21 logic


Rule:Ema9>ema21 buy and reverse shell . 
     We will use the 1 min bar here for
     initial tesing . This can be extended
     to any timeframe latter.

In [39]:
#Install WebSocket
!pip install websocket-client

In [40]:
!pip install alpaca-trade-api

In [41]:
#Import Libreris
import websocket 
import json
from datetime import datetime
import pandas as pd
import numpy as np
from IPython.core.display import JSON

In [42]:
#Map Google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [43]:
ticket_interested = (input('Enter The Stock yo wanted to autotrade')).upper()
print(ticket_interested+ ' Will be autotraded now!! Sitback and enjoy your day!!')

listen_message='{"action":"subscribe","bars":["'+ticket_interested+'"]}'
print(listen_message)

Enter The Stock yo wanted to autotradebtcusd
BTCUSD Will be autotraded now!! Sitback and enjoy your day!!
{"action":"subscribe","bars":["BTCUSD"]}


In [44]:
quantity_interested = float(input('Enter The Stock yo wanted to autotrade'))
quantity_interested

Enter The Stock yo wanted to autotrade0.1


0.1

In [45]:
#Import Credentials. Kept the credential in the file Cred.py under the dir below for security.
import sys
import os
import glob
print(glob.glob("/content/drive/*"))
os.chdir('/content/drive//MyDrive/AlpecaTrading/Config')
#sys.path.append('/content/drive/AlpecaTrading/Config')
print(os.getcwd())
from  Cred import *
print(API_KEY)
print(SECRET_KEY)

['/content/drive/MyDrive']
/content/drive/MyDrive/AlpecaTrading/Config
PKENE1R5LIQR1J3778D0
nT8XMPHULWGvTu4jnDJc6UMK2tSyUAc2YS7npTNV


In [46]:
####################TestCode To check the trading api9SDK working )#########################################
import alpaca_trade_api as tradeapi
paper_market='https://paper-api.alpaca.markets'
api = tradeapi.REST(key_id=API_KEY,secret_key=SECRET_KEY,base_url=paper_market)
acc=api.get_account()
acc

Account({   'account_blocked': False,
    'account_number': 'PA3WFYAGY0TD',
    'accrued_fees': '0',
    'buying_power': '154153.5082',
    'cash': '76852.4041',
    'created_at': '2022-02-04T18:16:52.455679Z',
    'crypto_status': 'ACTIVE',
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '100382.63028',
    'id': '818445eb-79e9-4861-86b3-ce3418a541cc',
    'initial_margin': '23305.87618',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '23530.22618',
    'maintenance_margin': '23095.38618',
    'multiplier': '2',
    'non_marginable_buying_power': '76679.9',
    'pattern_day_trader': False,
    'pending_transfer_in': '0',
    'portfolio_value': '100382.63028',
    'regt_buying_power': '154153.5082',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '0',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': Fals

In [110]:
position=json.loads(str(api.get_position(ticket_interested)).split('(')[1].split(')')[0].replace('\n','').replace("'",'"').replace('False','"F"').replace('True','"T"'))['qty']
if int(position) >= 1:
  print('Selling')
else:
  print('Buying')

Selling


In [48]:
#Websocket Code
#socket = "wss://stream.data.alpaca.markets/v2/sip" #Webscoket given by alpeca for 
#socket = "wss://paper-api.alpaca.markets/stream" #Webscoket given by alpeca for 
socket = "wss://stream.data.alpaca.markets/v1beta1/crypto"

print(socket.split('/')[-1])

def on_ope_fun(ws):
    print("Opened")
    auth_data = {"action": "auth","key": API_KEY,"secret": SECRET_KEY }  #Key and Secret getting from the Cred file. 
    ws.send(json.dumps(auth_data))
    #listen_message={"action":"subscribe","trades":["BTCUSD"],"quotes":["LTCUSD","ETHUSD"],"bars":["BCHUSD"]}
    #listen_message={"action":"subscribe","trades":["BTCUSD"]}
    #listen_message={"action":"subscribe","bars":["BTCUSD"]}
    listen_message='{"action":"subscribe","bars":["'+ticket_interested+'"]}'
    print(listen_message)
    #listen_message={"action":"subscribe","bars":["AAPL"]}  ### --> Modify based on whatever symbol you want to check
    ws.send(json.dumps(json.loads(listen_message)))


    print('Still Open')

def on_msg_fun(ws,message):
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("messgae received at"+ current_time)
    print(message)
    print('Strated wring to the steam file')
    trade_ema_9_21(message)
    print('Wrttin  steam file')

def on_clg_fun(ws):
    print("Closed")

crypto


/content/drive//MyDrive/AlpecaTrading/DataFiles/StreamFiles/AAPL.csv


In [49]:
#Trading Process 
def trade_ema_9_21(message):
  try:
    

    #######################Create a file which will hold the stream data based on the ticker name ########################################################
    messageJson = json.loads(message)
    df = pd.DataFrame(messageJson)
    df.to_csv('/content/drive//MyDrive/AlpecaTrading/DataFiles/StreamFiles/{}.csv'.format(ticket_interested),mode='a', index=False, header=False)



    with open('/content/drive//MyDrive/AlpecaTrading/DataFiles/StreamFiles/{}.csv'.format(ticket_interested), 'r') as fp:
      for count, line in enumerate(fp):
        pass
      total_lines =count+1

    print('Total Lines'+str(total_lines))


    if total_lines > 3:
      print('Calucation Started')

      if (socket.split('/')[-1] == 'sip'):

        #####################read the file for next set of operations#########################################################################################
        df = pd.read_csv('/content/drive//MyDrive/AlpecaTrading/DataFiles/StreamFiles/{}.csv'.format(ticket_interested),skiprows=3,index_col=False)
        df.columns =['Message Type', 
                'Symbol', 
                'Open', 
                'Close',
                'High',
                'Low',
                'Volume',
                'TimeStamp',
                'n',
                'vw']

      if (socket.split('/')[-1] == 'crypto'):


        #####################read the file for next set of operations#########################################################################################
        df = pd.read_csv('/content/drive//MyDrive/AlpecaTrading/DataFiles/StreamFiles/{}.csv'.format(ticket_interested),skiprows=3,index_col=False)
        df.columns =['Message Type', 
                'Symbol', 
                'Exchange', 
                'Open', 
                'Close',
                'High',
                'Low',
                'Volume',
                'TimeStamp',
                'n',
                'vw']

    if total_lines > 3:      

      # print(df)
      df_unique=df.drop_duplicates(subset=['TimeStamp']).copy()
      #print(df_unique.info())
      df_unique.loc[:,'TimeStamp'] = pd.to_datetime(df_unique['TimeStamp'],format='%Y-%m-%dT%H:%M:%SZ')
      #print(df_unique.info())
      df_unique.set_index('TimeStamp',inplace=True)      #print(df_unique)


      print('Total Lines in the unique file :'+ str(len(df_unique.index)))
      #print(df_unique)
      #print(df_unique)

      #####################Get the Exp Moving average for 9 and 21 minutes in the closed bar value##########################################################
      if(len(df_unique.index) > 5 ):
        print('------------------ 5 unique index----------------------') #process will start 1ith 21 points 


        df_unique['9minEMAClosed']  = df_unique['Close'].ewm(span=9,adjust=True).mean()
        df_unique['21minEMAClosed'] = df_unique['Close'].ewm(span=21,adjust=True).mean()
        print(df_unique[['Symbol','Close','9minEMAClosed','21minEMAClosed']])

        ###Write the file to tarck the EMA data .. Can plot using this data
        df_unique[['Symbol','Close','9minEMAClosed','21minEMAClosed']].to_csv('/content/drive//MyDrive/AlpecaTrading/DataFiles/EmaFiles/{}EMA.csv'.format(ticket_interested), index=True, header=True)
        #df_unique[['Close','9minEMAClosed','21minEMAClosed']].plot(kind='line',figsize=(16,6))
        if ( (df_unique.iloc[-1,3] > df_unique.iloc[-1,9]) and (df_unique.iloc[-1,9] > df_unique.iloc[-1,10]) ):
          print('-----------Buying-------------')## Buy one
          api.submit_order(
              symbol=ticket_interested,
              qty=quantity_interested,
              side='buy',
              type='market',
              time_in_force='day')

                  # api.submit_order(
        #     symbol='SPY',
        #     qty=1.5,  # fractional shares
        #     side='buy',
        #     type='market',
        #     time_in_force='day',
        #     )


        elif((df_unique.iloc[-1,3] < df_unique.iloc[-1,9]) and (df_unique.iloc[-1,10] > df_unique.iloc[-1,9]) ):
          print('-----------Selling------------')## Sell everything
          for pos in api.list_positions():
            if (pos.symbol == ticket_interested):
              total_qty =pos.qty
              print(total_qty)
              api.submit_order(
                  symbol=ticket_interested,
                  qty=total_qty,
                  side='sell',
                  type='market',
                  time_in_force='day')

  
    
    


        else :
          print('-----------stiing and wating to buy or sell------------')## Doing nothing





        # api.submit_order(
        #     symbol='SPY',
        #     qty=1.5,  # fractional shares
        #     side='buy',
        #     type='market',
        #     time_in_force='day',
        #     )





# #####################Get the Exp Moving average for 9 and 21 minutes in the closed bar value##########################################################
    # if(len(df.index) > 21 ):
    #   df_unique['9minEMAClosed']  = df_unique['Close'].ewm(span=9,adjust=True).mean()
    #   df_unique['21minEMAClosed'] = df_unique['Close'].ewm(span=21,adjust=True).mean()

    #   print(df_unique[['9minEMAClosed','21minEMAClosed']])


    #   print('Calculation started as 21 data points avelable')
    
    # else:
    #   print('Wating for 21 data points ')





    # #####################read the file for next set of operations#########################################################################################
    # df = pd.read_csv('/content/drive//MyDrive/AlpecaTrading/DataFiles/StreamFiles/{}.csv',skiprows=3,index_col=False)
    # df.columns =['Message Type', 
    #          'Symbol', 
    #          'Open', 
    #          'Close',
    #          'High',
    #          'Low',
    #          'Volume',
    #          'TimeStamp',
    #          'n',
    #          'vw']
    # #print(df)
    # df_unique=df.drop_duplicates(subset=['TimeStamp']).copy()
    # #print(df_unique.info())
    # df_unique.loc[:,'TimeStamp'] = pd.to_datetime(df_unique['TimeStamp'],format='%Y-%m-%dT%H:%M:%SZ')
    # #print(df_unique.info())
    # df_unique.set_index('TimeStamp',inplace=True)
    # #print(df_unique)



    # #####################Get the Exp Moving average for 9 and 21 minutes in the closed bar value##########################################################
    # if(len(df.index) > 21 ):
    #   df_unique['9minEMAClosed']  = df_unique['Close'].ewm(span=9,adjust=True).mean()
    #   df_unique['21minEMAClosed'] = df_unique['Close'].ewm(span=21,adjust=True).mean()

    #   print(df_unique[['9minEMAClosed','21minEMAClosed']])


    #   print('Calculation started as 21 data points avelable')
    
    # else:
    #   print('Wating for 21 data points ')




  except Exception as ex:
    print(ex)

    



In [50]:
ws = websocket.WebSocketApp(socket, on_open=on_ope_fun,on_message=on_msg_fun)
ws.run_forever()

Opened
{"action":"subscribe","bars":["BTCUSD"]}
Still Open
messgae received at04:23:51
[{"T":"success","msg":"connected"}]
Strated wring to the steam file
Total Lines1
Wrttin  steam file
messgae received at04:23:51
[{"T":"success","msg":"authenticated"}]
Strated wring to the steam file
Total Lines2
Wrttin  steam file
messgae received at04:23:51
[{"T":"subscription","trades":[],"quotes":[],"bars":["BTCUSD"],"dailyBars":[]}]
Strated wring to the steam file
Total Lines3
Wrttin  steam file
messgae received at04:24:00
[{"T":"b","S":"BTCUSD","x":"CBSE","o":41672.36,"c":41693.44,"h":41706.06,"l":41670.7,"v":10.48475222,"t":"2022-02-05T04:23:00Z","n":472,"vw":41687.4369592353}]
Strated wring to the steam file
Total Lines4
Calucation Started
Total Lines in the unique file :0
Wrttin  steam file
messgae received at04:24:00
[{"T":"b","S":"BTCUSD","x":"FTXU","o":41671,"c":41676,"h":41676,"l":41671,"v":0.1722,"t":"2022-02-05T04:23:00Z","n":3,"vw":41672.6840882695}]
Strated wring to the steam file
To

False

#----------------------------------Code End-----------------------------
###All Below this is scratch pad for tesing 

In [24]:
          api.submit_order(
              symbol=ticket_interested,
              qty=quantity_interested,
              side='buy',
              type='market',
              time_in_force='day')

Order({   'asset_class': 'crypto',
    'asset_id': 'ef145fe0-95cd-453a-8609-db7200ff0279',
    'canceled_at': None,
    'client_order_id': 'ce35a5f1-d1d2-432c-9bf2-252173414743',
    'created_at': '2022-02-05T02:53:15.918727838Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': '44a5a737-1efc-4fcd-8c93-20cbaa2fc042',
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '0.01',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2022-02-05T02:53:15.915777584Z',
    'symbol': 'BTCUSD',
    'time_in_force': 'day',
    'trail_percent': None,
    'trail_price': None,
    'type': 'market',
    'updated_at': '2022-02-05T02:53:15.918727838Z'})

In [37]:
for pos in api.list_positions():
  if (pos.symbol == ticket_interested):
    total_qty =pos.qty
    print(total_qty)



0.551


In [122]:
df = pd.read_csv('/content/drive//MyDrive/AlpecaTrading/DataFiles/StreamFiles/AAPL.csv'.format(ticket_interested),skiprows=3,index_col=False)
df.columns =['Message Type', 
                'Symbol', 
                'Open', 
                'Close',
                'High',
                'Low',
                'Volume',
                'TimeStamp',
                'n',
                'vw']

In [ ]:
'/content/drive//MyDrive/AlpecaTrading/DataFiles/StreamFiles/{}.csv'.format(ticket_interested),mode='a', index=False, header=False

In [162]:
      df_unique=df.drop_duplicates(subset=['TimeStamp']).copy()
      #print(df_unique.info())
      df_unique.loc[:,'TimeStamp'] = pd.to_datetime(df_unique['TimeStamp'],format='%Y-%m-%dT%H:%M:%SZ')
      #print(df_unique.info())
      df_unique.set_index('TimeStamp',inplace=True)      #print(df_unique)


In [165]:
        df_unique['9minEMAClosed']  = df_unique['Close'].ewm(span=9,adjust=True).mean()
        df_unique['21minEMAClosed'] = df_unique['Close'].ewm(span=21,adjust=True).mean()
        print(df_unique[['Symbol','Close','9minEMAClosed','21minEMAClosed']])


df_unique[['Symbol','Close','9minEMAClosed','21minEMAClosed']].to_csv('/content/drive//MyDrive/AlpecaTrading/DataFiles/EmaFiles/{}EMA.csv'.format(ticket_interested),mode='a', index=True, header=True)

                    Symbol     Close  9minEMAClosed  21minEMAClosed
TimeStamp                                                          
2022-02-04 20:34:00   AAPL  173.9600     173.960000      173.960000
2022-02-04 20:35:00   AAPL  173.8150     173.879444      173.884048
2022-02-04 20:36:00   AAPL  173.8850     173.881721      173.884396
2022-02-04 20:37:00   AAPL  173.9420     173.902141      173.900916
2022-02-04 20:38:00   AAPL  173.8300     173.880681      173.883909
2022-02-04 20:39:00   AAPL  173.7300     173.839838      173.851783
2022-02-04 20:40:00   AAPL  173.6359     173.788227      173.811471
2022-02-04 20:41:00   AAPL  173.4300     173.702138      173.746467
2022-02-04 20:42:00   AAPL  173.5250     173.661218      173.711507


In [157]:
if(len(df_unique.index) > 5 ):
  print('------------------ 5 unique index----------------------') #process will start 1ith 21 points 
  df_unique['9minEMAClosed']  = df_unique['Close'].ewm(span=9,adjust=True).mean()
  df_unique['21minEMAClosed'] = df_unique['Close'].ewm(span=21,adjust=True).mean()
  print(df_unique)
  if ( df_unique.iloc[-1,3] > df_unique.iloc[-1,9]  ):
    print('-----------Buying-------------')
  else:
    print('-----------Selling------------')



        
else:
  print('-----------------less than 5 -------------------------')


------------------ 5 unique index----------------------
                    Message Type Symbol  ...  9minEMAClosed  21minEMAClosed
TimeStamp                                ...                               
2022-02-04 20:34:00            b   AAPL  ...     173.960000      173.960000
2022-02-04 20:35:00            b   AAPL  ...     173.879444      173.884048
2022-02-04 20:36:00            b   AAPL  ...     173.881721      173.884396
2022-02-04 20:37:00            b   AAPL  ...     173.902141      173.900916
2022-02-04 20:38:00            b   AAPL  ...     173.880681      173.883909
2022-02-04 20:39:00            b   AAPL  ...     173.839838      173.851783
2022-02-04 20:40:00            b   AAPL  ...     173.788227      173.811471
2022-02-04 20:41:00            b   AAPL  ...     173.702138      173.746467
2022-02-04 20:42:00            b   AAPL  ...     173.661218      173.711507

[9 rows x 11 columns]
-----------Selling------------


In [156]:
  # df_unique.iloc[-1] 
  if ( df_unique.iloc[-1,3] > df_unique.iloc[-1,9]  ):
    print('-----------Buying-------------')
  else:
    print('-----------Selling------------')

-----------Selling------------


In [153]:
df_unique.iloc[-1,10]

173.7115071870725

In [124]:
df_unique

,Message Type,Symbol,Open,Close,High,Low,Volume,n,vw
TimeStamp,,,,,,,,,
2022-02-04 20:34:00,b,AAPL,173.9600,173.9600,174.055,173.890,156822,1657,173.974373
2022-02-04 20:35:00,b,AAPL,173.9600,173.8150,173.970,173.800,180827,1688,173.847236
2022-02-04 20:36:00,b,AAPL,173.8000,173.8850,173.970,173.770,193547,1945,173.864692
2022-02-04 20:37:00,b,AAPL,173.8900,173.9420,173.970,173.860,138235,1367,173.922554
2022-02-04 20:38:00,b,AAPL,173.9500,173.8300,174.000,173.750,176090,1904,173.898841
2022-02-04 20:39:00,b,AAPL,173.8308,173.7300,173.835,173.700,155259,1571,173.758775
2022-02-04 20:40:00,b,AAPL,173.7300,173.6359,173.755,173.581,187127,1970,173.688963
2022-02-04 20:41:00,b,AAPL,173.6300,173.4300,173.640,173.370,259699,2115,173.476760
2022-02-04 20:42:00,b,AAPL,173.4300,173.5250,173.560,173.370,208693,1755,173.473676


In [115]:
trade_ema_9_21('Test')


Expecting value: line 1 column 1 (char 0)


In [116]:
ws = websocket.WebSocketApp(socket, on_open=on_ope_fun,on_message=on_msg_fun)
ws.run_forever()

Opened
{"action":"subscribe","bars":["AAPL"]}
Still Open
messgae received at20:33:46
[{"T":"success","msg":"connected"}]
Strated wring to the steam file
Total Lines1
Wrttin  steam file
messgae received at20:33:47
[{"T":"success","msg":"authenticated"}]
Strated wring to the steam file
Total Lines2
Wrttin  steam file
messgae received at20:33:47
[{"T":"subscription","trades":[],"quotes":[],"bars":["AAPL"],"dailyBars":[],"statuses":[],"lulds":[],"corrections":[],"cancelErrors":[]}]
Strated wring to the steam file
Total Lines3
Wrttin  steam file
messgae received at20:34:00
[{"T":"b","S":"AAPL","o":173.81,"c":173.9699,"h":174.1,"l":173.81,"v":377468,"t":"2022-02-04T20:33:00Z","n":3540,"vw":173.976803}]
Strated wring to the steam file
Total Lines4
Calucation Started
Total Lines in the unique file :0
Wrttin  steam file
messgae received at20:35:03
[{"T":"b","S":"AAPL","o":173.96,"c":173.96,"h":174.055,"l":173.89,"v":156822,"t":"2022-02-04T20:34:00Z","n":1657,"vw":173.974373}]
Strated wring to t

False